In [43]:
import os
import sys
import time
import pickle

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
import numpy as np

import allennlp
# from allennlp.common.testing import AllenNlpTestCase, ModelTestCase
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
# from Module import 

In [44]:
raw = pickle.load(open('rnn_input.txt', 'rb'))

In [45]:
print(raw[0:2])

[{'verbs': (('worked_with',), ['shortened', 'stylized_as', 'is', 'is_owned_by']), 'label': 'SUPPORTS'}, {'verbs': (('worked_with',), ['played', 'appearing_as', 'intended_as']), 'label': 'SUPPORTS'}]


In [46]:
x_train, y_train = [], []

In [47]:
for d in raw:
    x_train.append(d['verbs'])
    y_train.append(d['label'])

In [48]:
print(y_train[:100])

['SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'REFUTES', 'REFUTES', 'REFUTES', 'REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', '

In [49]:
y_train = np.array(y_train)
print(y_train[:20])

['SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'REFUTES'
 'NOT ENOUGH INFO' 'SUPPORTS' 'SUPPORTS' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS']


In [50]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)
print(integer_encoded[:20])

[2 2 2 2 2 1 0 2 2 0 0 2 2 2 2 2 2 2 2 2]


In [52]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded[:20])

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [53]:
print(x_train[0])

(('worked_with',), ['shortened', 'stylized_as', 'is', 'is_owned_by'])


In [54]:
clf = SVC(gamma='auto')

In [86]:
import pickle
from nltk.tokenize import word_tokenize
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, LabelField
from typing import Iterator, List, Dict
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data import Vocabulary
from allennlp.data.dataset import Batch


class VerbDatasetReader(DatasetReader):

    def __init__(self,sentence_indexers:Dict[str,TokenIndexer]=None )-> None:
        super().__init__(lazy=False)
        self.sentence_indexers=sentence_indexers or {"sentence":SingleIdTokenIndexer()}

    def text_to_instance(self, claim: List[List], evidence: List[List], labels:str = None) -> Instance:
        evidence_tokenized, claim_tokenized = [], []
        for c in claim:
            for word in word_tokenize(c):
                claim_tokenized.append(Token(word))
        for e in evidence:
            for word in word_tokenize(e):
                evidence_tokenized.append(Token(word))
        claim_textfiled = TextField(claim_tokenized, self.sentence_indexers)
        evidence_textfiled = TextField(evidence_tokenized, self.sentence_indexers)
        fields={'claim':claim_textfiled, 'evidence':evidence_textfiled,'label': LabelField(labels)}
        return Instance(fields)

    def _read(self, file_path: str)->Iterator[Instance]:
        mlinput_merge=pickle.load(open(file_path,'rb'))
        for entry in mlinput_merge:
            claim = [entry['claim']]
            evidence = []
            for sentence in entry['evidence']:
                sentence = ' '.join(sentence)
                evidence.append(sentence)
            yield self.text_to_instance(claim, evidence, entry['label'])



reader=VerbDatasetReader()

instances = reader.read('mlinput_merge.txt')







0it [00:00, ?it/s]
1it [00:00,  2.40it/s]
105it [00:00,  3.42it/s]
237it [00:00,  4.89it/s]
350it [00:00,  6.97it/s]
474it [00:00,  9.93it/s]
608it [00:00, 14.14it/s]
740it [00:01, 20.10it/s]
867it [00:01, 28.53it/s]
995it [00:01, 40.37it/s]
1124it [00:01, 56.90it/s]
1247it [00:01, 79.64it/s]
1373it [00:01, 110.77it/s]
1502it [00:01, 152.63it/s]
1633it [00:01, 207.47it/s]
1759it [00:01, 276.45it/s]
1885it [00:01, 360.87it/s]
2013it [00:02, 459.74it/s]
2142it [00:02, 569.43it/s]
2282it [00:02, 689.55it/s]
2412it [00:02, 797.51it/s]
2541it [00:02, 890.24it/s]
2680it [00:02, 996.69it/s]
2810it [00:02, 1037.01it/s]
2938it [00:02, 1095.78it/s]
3080it [00:02, 1175.32it/s]
3224it [00:02, 1243.39it/s]
3362it [00:03, 1279.28it/s]
3498it [00:03, 1294.49it/s]
3633it [00:03, 1295.00it/s]
3767it [00:03, 1248.62it/s]
3895it [00:03, 1253.80it/s]
4023it [00:03, 1209.70it/s]
4151it [00:03, 1229.31it/s]
4276it [00:03, 1184.69it/s]
4407it [00:03, 1216.80it/s]
4538it [00:04, 1242.30it/s]
4674it [00:04, 1

35544it [00:27, 1367.87it/s]
35682it [00:27, 1358.12it/s]
35819it [00:27, 1353.58it/s]
35955it [00:27, 1322.58it/s]
36093it [00:27, 1338.15it/s]
36241it [00:27, 1377.48it/s]
36380it [00:28, 1310.95it/s]
36516it [00:28, 1324.18it/s]
36650it [00:28, 1300.48it/s]
36789it [00:28, 1323.57it/s]
36923it [00:28, 1314.17it/s]
37055it [00:28, 1296.97it/s]
37186it [00:28, 1278.10it/s]
37315it [00:28, 1265.02it/s]
37455it [00:28, 1301.72it/s]
37586it [00:28, 1294.38it/s]
37716it [00:29, 1290.13it/s]
37862it [00:29, 1335.67it/s]
37997it [00:29, 1339.03it/s]
38132it [00:29, 1309.72it/s]
38264it [00:29, 1296.53it/s]
38394it [00:29, 1290.72it/s]
38525it [00:29, 1295.60it/s]
38661it [00:29, 1310.86it/s]
38793it [00:29, 1310.87it/s]
38925it [00:29, 1305.76it/s]
39056it [00:30, 1256.02it/s]
39184it [00:30, 1258.11it/s]
39311it [00:30, 1237.73it/s]
39457it [00:30, 1296.23it/s]
39602it [00:30, 1335.51it/s]
39741it [00:30, 1348.83it/s]
39877it [00:30, 1326.12it/s]
40016it [00:30, 1343.39it/s]
40151it [00:30

In [91]:
vars(instances[0].fields['evidence'])

{'tokens': [The,
  Fox,
  Broadcasting,
  Company,
  -LRB-,
  often,
  shortened,
  to,
  Fox,
  and,
  stylized,
  as,
  FOX,
  -RRB-,
  is,
  an,
  American,
  English,
  language,
  commercial,
  broadcast,
  television,
  network,
  that,
  is,
  owned,
  by,
  the,
  Fox,
  Entertainment,
  Group,
  subsidiary,
  of,
  21st,
  Century,
  Fox,
  .,
  He,
  then,
  played,
  Detective,
  John,
  Amsterdam,
  in,
  the,
  short-lived,
  Fox,
  television,
  series,
  New,
  Amsterdam,
  -LRB-,
  2008,
  -RRB-,
  ,,
  as,
  well,
  as,
  appearing,
  as,
  Frank,
  Pike,
  in,
  the,
  2009,
  Fox,
  television,
  film,
  Virtuality,
  ,,
  originally,
  intended,
  as,
  a,
  pilot,
  .],
 '_token_indexers': {'sentence': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x1348908d0>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

In [92]:
vocab = Vocabulary.from_instances(instances)

100%|██████████| 145449/145449 [00:06<00:00, 22157.31it/s]


In [100]:
# Basic embeddings
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

claim_embedding = Embedding(num_embeddings=vocab.get_vocab_size("claim"), embedding_dim=100, padding_index=0)
evidence_embedding = Embedding(num_embeddings=vocab.get_vocab_size("evidence"), embedding_dim=100, padding_index=0)

# This is how the text_field_embedder knows which function to apply to which array. 
# There should be a 1-1 mapping between TokenIndexers and TokenEmbedders in your model.
text_field_embedder = BasicTextFieldEmbedder({"claim": claim_embedding, "evidence": evidence_embedding})

# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

AttributeError: module 'torch' has no attribute '_six'

In [61]:
print(train_dataset[0])

Instance with fields:
 	 evidece: TextField of length 85 with text: 
 		[Nikolaj, Coster-Waldau, worked, with, the, Fox, Broadcasting, Company, ., The, Fox, Broadcasting,
		Company, -LRB-, often, shortened, to, Fox, and, stylized, as, FOX, -RRB-, is, an, American, English,
		language, commercial, broadcast, television, network, that, is, owned, by, the, Fox, Entertainment,
		Group, subsidiary, of, 21st, Century, Fox, ., He, then, played, Detective, John, Amsterdam, in, the,
		short-lived, Fox, television, series, New, Amsterdam, -LRB-, 2008, -RRB-, ,, as, well, as,
		appearing, as, Frank, Pike, in, the, 2009, Fox, television, film, Virtuality, ,, originally,
		intended, as, a, pilot, .]
 		and TokenIndexers : {'sentence': 'SingleIdTokenIndexer'} 
 	 label: LabelField with label: SUPPORTS in namespace: 'labels'.' 



In [70]:
with open('mlinput_merge.txt', 'rb') as m:
    a = pickle.load(m)
    print(a[0])



{'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'label': 'SUPPORTS', 'evidence': [['The', 'Fox', 'Broadcasting', 'Company', '-LRB-', 'often', 'shortened', 'to', 'Fox', 'and', 'stylized', 'as', 'FOX', '-RRB-', 'is', 'an', 'American', 'English', 'language', 'commercial', 'broadcast', 'television', 'network', 'that', 'is', 'owned', 'by', 'the', 'Fox', 'Entertainment', 'Group', 'subsidiary', 'of', '21st', 'Century', 'Fox', '.\n'], ['He', 'then', 'played', 'Detective', 'John', 'Amsterdam', 'in', 'the', 'short-lived', 'Fox', 'television', 'series', 'New', 'Amsterdam', '-LRB-', '2008', '-RRB-', ',', 'as', 'well', 'as', 'appearing', 'as', 'Frank', 'Pike', 'in', 'the', '2009', 'Fox', 'television', 'film', 'Virtuality', ',', 'originally', 'intended', 'as', 'a', 'pilot', '.\n']]}


In [ ]:
class TestSemanticScholarDatasetReader(AllenNlpTestCase):
    def test_read_from_file(self):
        reader = SemanticScholarDatasetReader()
        dataset = reader.read('tests/fixtures/s2_papers.jsonl')